In [1]:
import pandas as pd

In [3]:
%run setup.ipynb

In [14]:
predictions = pd.read_csv(f"../data/predictions/predictions-{11}-{6}.csv")


In [15]:
store_sales = pd.concat([train, predictions])

In [16]:
predictions

,Unnamed: 0,Unnamed: 0.1,store_item,sales,autoreg,exp_smooth,exp_smooth_multi,ardl,prophet,xgb_preds
0,0,2016-07-03,1-1,35,33.564926,34.968360,37.358205,28.270421,31.430007,42.183426
1,1,2016-07-04,1-1,18,25.371284,27.868874,29.091349,21.079731,23.552477,20.786821
2,2,2016-07-05,1-1,31,27.200800,26.861003,27.695462,21.245674,26.190409,23.284859
3,3,2016-07-06,1-1,22,28.000816,28.344893,29.004482,27.543148,26.846605,26.260480
4,4,2016-07-07,1-1,33,26.200584,25.971329,26.564866,29.099083,27.095248,26.110456
...,...,...,...,...,...,...,...,...,...,...
272401,272401,2017-12-27,10-50,63,54.036777,70.629780,65.336560,52.203080,60.990158,59.709023
272402,272402,2017-12-28,10-50,59,48.369474,64.050097,56.627964,60.144470,64.923885,66.095240
272403,272403,2017-12-29,10-50,74,41.368979,56.527002,50.323118,60.437527,69.576875,70.068820
272404,272404,2017-12-30,10-50,62,43.701682,59.522268,52.400913,69.928363,74.064814,69.177450
